In [1]:
import json
import pandas as pd

with open('val_v1.0.json') as f:
    data = json.load(f)

data = data['data']
df = pd.DataFrame(data)
df

,questionId,question,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split
0,49153,"What is the ‘actual’ value per 1000, during th...",documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val
1,24580,What is name of university?,documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of Calif...",val
2,57349,What is the name of the company?,documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val
3,24581,Where is the university located ?,documents/nkbl0226_1.png,7027,nkbl0226,1,"[san diego, San Diego]",val
4,24582,To whom is the document sent?,documents/nkbl0226_1.png,7027,nkbl0226,1,[Paul],val
...,...,...,...,...,...,...,...,...
5344,24564,who is the vice president and secretary ?,documents/ntnk0226_15.png,7020,ntnk0226,15,"[Raymond C. Guth, Raymond c. Guth]",val
5345,49146,What is the ‘title’ of the plot?,documents/pybv0228_81.png,14465,pybv0228,81,[Age adjusted motor vehicle accident mortality...,val
5346,49147,What is ‘figure C.2.’?,documents/pybv0228_81.png,14465,pybv0228,81,[age adjusted motor vehicle accident mortality...,val
5347,49150,What is the maximum value at x axis?,documents/pybv0228_81.png,14465,pybv0228,81,[1980],val


In [2]:
# select 2200 random rows from the table
df = df[["question", "image", "answers"]]
df["answer"] = df["answers"].apply(lambda x: x[0])
df = df.sample(n=2200, random_state=42)
# convert all rows to strings
df["question"] = df["question"].astype(str)
df["answer"] = df["answer"].astype(str)
df["image"] = df["image"].astype(str)
df[:200].to_csv('seed.csv', index=False)
df[200:].to_csv('test.csv', index=False)

/var/folders/7q/qgypwzk535x3f_wvxqdpdbz40000gp/T/ipykernel_61363/4099873394.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["answer"] = df["answers"].apply(lambda x: x[0])


In [3]:
from autolabel import LabelingAgent, AutolabelConfig, AutolabelDataset

In [4]:
config = {
    "task_name": "DocumentQuestionAnswering",
    "task_type": "question_answering",
    "dataset": {"delimiter": ",", "label_column": "answer"},
    "model": {"provider": "openai", "name": "gpt-3.5-turbo"},
    "prompt": {
        "task_guidelines": "You are an expert at reading documents and understanding their content. Your task is to answer questions about the content of the document. Use the document when answering the question. Do not rely on outside knowledge.",
        "example_template": "Document Text: {text}\nQuestion: {question}\nAnswer: {answer}",
        "few_shot_examples": "seed.csv",
        "few_shot_selection": "semantic_similarity",
        "few_shot_num": 3,
    },
    "transforms": [
        {
            "name": "image",
            "output_columns": {"content_column": "text", "metadata_column": "metadata"},
            "params": {"file_path_column": "image", "lang": "eng"},
        }
    ],
}

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
config = AutolabelConfig(config)

In [7]:
agent = LabelingAgent(config)

seed_dataset = AutolabelDataset("seed.csv", config)
seed_dataset = agent.transform(seed_dataset)
seed_dataset.df.to_csv("seed.csv", index=False)

dataset = AutolabelDataset("test.csv", config, max_items=100)
dataset = agent.transform(dataset)
dataset.df.to_csv("test.csv", index=False)

Output()

Output()

In [8]:
agent = LabelingAgent(config)
agent.plan(dataset)

['question', 'image', 'answers', 'text', 'metadata']
["['Depth (inches)', 'Depth']", 'documents/rmwn0226_155.png', "{'format': 'PNG', 'mode': 'L', 'size': (2200, 1700), 'width': 2200, 'height': 1700, 'exif': None}", 'What is mentioned in the Y axis of this graph?', 'Depth (inches)\n\nCutting Longevity Test\n\n0.1 ms\n\n10 20 30 40 50\n\nNumber of cuts\n\n@ = eCurrent DePuy Bridgeback 410 Current DePuy Bridgeback Modified to 420\n\nDM\nSource: https://www.industrydocuments.ucsf.edu/docs/rmwn0226\n\nP-09686-00155\nDEPUY023042075\n']


TypeError: sequence item 4: expected str instance, float found